<a href="https://colab.research.google.com/github/vinesh1213/PROJECT.PY/blob/main/AIchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio>=4.44.0

In [2]:

!pip install python-dotenv>=1.0.1
!pip install openai>=1.40.0

In [ ]:
from __future__ import annotations
import os
from dotenv import load_dotenv
from typing import List, Tuple
import gradio as gr # Import gradio and alias as gr


# Placeholder definitions - Replace with your actual implementations
class Memory:
    def __init__(self, max_turns: int):
        self.max_turns = max_turns
        self.turns = []

    def add(self, role: str, content: str):
        self.turns.append({'role': role, 'content': content})
        if len(self.turns) > self.max_turns:
            self.turns.pop(0) # Simple FIFO for exceeding max_turns

    def as_messages(self):
        return self.turns

    def export_txt(self) -> str:
        return "\n".join(("You: " + t['content']) if t['role'] == "user" else ("CalmCare: " + t['content']) for t in self.turns)


class LLM:
    def __init__(self, api_key: str | None):
        self.api_key = api_key
        if not self.api_key:
            print("OpenAI API key not found. Using rule-based fallback.")

    def generate(self, messages: list, user_message: str) -> str:
        if not self.api_key:
            # Simple rule-based fallback
            if "hello" in user_message.lower():
                return "Hello there! How can I help you today?"
            elif "how are you" in user_message.lower():
                return "I am a chatbot, I don't have feelings, but I'm here to help you."
            elif "sad" in user_message.lower() or "unhappy" in user_message.lower():
                return "I'm sorry to hear that you're feeling sad. Would you like to talk about it."
            else:
                return "I'm here to listen if you'd like to share what's on your mind."
        else:
            # Replace with your actual OpenAI API call
            return f"LLM response to: {user_message}" # Placeholder for actual LLM call


# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# ---- Gradio UI ----------------------------------------------------------------
llm = LLM(OPENAI_API_KEY)


def chat_fn(message: str, history: List[Tuple[str, str]]):
    # Sync history → messages
    mem_local = Memory(max_turns=24)
    for u, a in history:
        if u:
            mem_local.add("user", u)
        if a:
            mem_local.add("assistant", a)
    mem_local.add("user", message or "")
    reply = llm.generate(mem_local.as_messages(), message or "")
    mem_local.add("assistant", reply)
    return reply


with gr.Blocks(title="CalmCare – Mental Health Chatbot", analytics_enabled=False) as demo:
    gr.Markdown(
        "# CalmCare – Mental Health Chatbot\n"
        "*Supportive, empathetic conversation with a safety-first design.*\n\n"
        "**Important:** This tool can’t provide diagnosis or replace professional care.\n"
        "If you may act on thoughts of harming yourself or someone else, contact your local emergency number."
    )
    gr.Markdown(
        f"Backend: {'OpenAI' if OPENAI_API_KEY else 'Rule‑based fallback'} — This is not medical advice."
    )


    chatbot = gr.Chatbot(label="CalmCare", height=450, type="tuples", show_copy_button=True)


    with gr.Row():
        msg = gr.Textbox(placeholder="Tell me what's on your mind…", label="Your message", autofocus=True)
        with gr.Row():
            send_btn = gr.Button("Send", variant="primary")
            clear_btn = gr.Button("Clear Chat")
            export_btn = gr.Button("Export Conversation (.txt)")


    def _user_send(user_message, chat_history):
        if not user_message:
            return gr.update(), chat_history
        response = chat_fn(user_message, chat_history)
        chat_history = chat_history + [(user_message, response)]
        return "", chat_history


    send_btn.click(_user_send, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(_user_send, inputs=[msg, chatbot], outputs=[msg, chatbot])

    demo.launch(debug=True)

OpenAI API key not found. Using rule-based fallback.


/tmp/ipython-input-2025061705.py:83: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="CalmCare", height=450, type="tuples", show_copy_button=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://713baf2b7f455e6fb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
